## Combining symbolic regression and Cox regression improves prediction of heart failure deaths

Heart failure is a clinical syndrome characterised by a reduced ability of the heart to pump or fill with blood. This leads to fatigue, shortness of breath and poor quality of life. Patients with heart failure have a high mortality rate, and various biostatistical methods as well as machine learning methods have been applied to predict heart failure deaths from patients' medical records.

In this study, we demonstrate that using symbolic regression to find simple mathematical functions of covariates may improve the ability to predict death due to heart failure compared to existing methods. We use a newly invented symbolic regression method called the QLattice to analyse a data set of medical records for 299 Pakistani patients with heart failure. We use the QLattice to find a minimal set of mathematical functional transformations of the available covariates. We then use Cox regression to model survival based on these transformed covariates rather than the covariates themselves.

In [ ]:
import pandas as pd
import numpy as np
import feyn
from lifelines import CoxPHFitter

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
import lifelines
lifelines.__version__

In [ ]:
feyn.__version__

Heart failure (HF) is a clinical syndrome characterised by a reduction in the ability of the heart to pump or fill with blood. Physiologically, HF can be defined as an inadequate cardiac output to meet metabolic demands, often manifesting as increased left ventricular filling pressure.\cite{Tan2010}

Among the causes of HF are coronary heart disease, hypertension, diabetes, obesity, and smoking\cite{Virani2020}. HF affects at least 26 million people globally and has high mortality and morbidity\cite{Savarese2017}.

Various methods have been developed to estimate the risk of death for patients with HF. Well-known models include the Seattle Heart Failure Model\cite{Levy2006} and the ADHERE model\cite{Fonarow2005}. Although these models are accurate, they are unintuitive and rely on extensive medical records, making them hard to apply in a clinical setting.

Ahmad et al. published a study of 299 patients with HF admitted to Faisalabad Institute of Cardiology or Allied Hospital Faisalabad, Punjab, Pakistan.\cite{Ahmad2017}. In their study, Ahmad et al. used Cox regression to model survival of patients with much fewer covariates than in the Seattle Heart Failure Model and ADHERE. 

The authors chose to make the data set freely available, and it has subsequently been used in additional analyses using both biostatistical\cite{Zahid2019} and machine learning techniques\cite{Chicco2020}.

## Read in the Ahmad dataset
We also rename "Ejection.Fraction" to "EF" to allow it to be used in equations

In [ ]:
df = pd.read_csv("S1Data.csv")
df.rename({
    "Ejection.Fraction": "EF"
}, axis=1, inplace=True)


## Balance

Check the balance of the dataset. (It is balanced enough to be used without sample weights)

In [ ]:
df["Event"].value_counts()

## Inspect the first few rows

In [ ]:
df
# print(df.describe().T.to_latex(float_format="%.2f"))

## Desriptive statistics

In [ ]:
df.describe().T

## Connect to QLattice

The QLattice is a symbolic regressor inspired by quantum field theory\cite{feyn}. The QLattice runs on a dedicated high-performance computing cluster and models the---in principle infinite---list of mathematical expressions as a superposition of an infinite set of spatial paths. The QLattice searches the space of all functional forms, including parameters, that will best model the output from the input. The result of the search is a list of mathematical expressions sorted by how well they match observations.

Note: The QLattice is an actual simulator and a limited resource. To run this code you need to obtain access to one. See www.abzu.ai or contact the author. Abzu *does* provide QLattices free of charge for scientific purposes given a reasonable description of the project.



In [ ]:
ql = feyn.QLattice()
ql.reset()

# Roam Free

First we will search for a model that best describes the probability of death given three of the available inputs. The purpose of this search is to identify three important covariates that we then examine further. Other feature selection methods have been used on the same data set by Zahid et al.\cite{Zahid2019} and by Chicco and Jurman.\cite{Chicco2020}. We choose to use the QLattice to search for important covariates, because that ensures that the covariates selected are those that best predict death events given a combination of the elementary functions available.

The QLattice freely searches through all the available input covariates, combines them in different ways using the elementary mathematical functions and tests how the resulting expression compares to the actual observations. Even given the limit of three inputs, the QLattice will search through millions of different expressions to come up with the suggested expression.

### FIltering:
To ensure that the QLattice uses only three covariates, we enforce a limit on the number of edges in the graph to 5.

We also restrict the elementary functions allowed, mainly to rule of the flexible but not very interpretable tanh and gaussian functions.

In [ ]:
qg_all = ql.get_classifier(df.columns.drop(["TIME"]), output="Event") \
    .filter(feyn.filters.MaxEdges(5)) \
    .filter(feyn.filters.Functions(["log","exp","inv","sqrt","add","multiply"]))

In [ ]:
for _ in range(10):
    qg_all.fit(df, threads=7)
    ql.update(qg_all.best())

In [ ]:
qg_all[0].sympify(2)

In [ ]:
qg_all[0].plot_roc_curve(df)


## Selected features

In this study we will not analyse this specific model any further, except by noticing that the three features chosen by the QLattice when freely searching for symbolic forms are *ejection fraction*, *serum creatinine* and *age*. This is consistent with the results from other studies of the same data set using different feature selection methods\cite{Zahid2019,Chicco2020}.

# How does EF relate to death?

Ues the QLattice to search for the best fitting mathematical relation between the ejection fraction and the probability of death

In [ ]:
qg_ef = ql.get_classifier(["EF"], output="Event") \
    .filter(feyn.filters.MaxEdges(2)) \
    .filter(feyn.filters.Functions(["log","exp","inv","sqrt"]))

In [ ]:
for _ in range(6):
    qg_ef.fit(df, threads=7)
    ql.update(qg_ef.best())

In [ ]:
qg_ef[0].sympify(2)

In [ ]:
qg_ef[0].plot_roc_curve(df)

# How does serum creatinine relate to death?

Ues the QLattice to search for the best fitting mathematical relation between the serum creatinine and the probability of death

In [ ]:
qg_sc = ql.get_classifier(["Creatinine"], output="Event") \
    .filter(feyn.filters.MaxEdges(2)) \
    .filter(feyn.filters.Functions(["log","exp","inv","sqrt"]))

In [ ]:
for _ in range(6):
    qg_sc.fit(df, threads=7)
    ql.update(qg_sc.best())

In [ ]:
qg_sc[0].sympify(2)

In [ ]:
qg_sc[0].plot_roc_curve(df)

# How does age relate to death?

Ues the QLattice to search for the best fitting mathematical relation between age and the probability of death

In [ ]:
qg_age = ql.get_classifier(["Age"], output="Event") \
    .filter(feyn.filters.MaxEdges(2)) \
    .filter(feyn.filters.Functions(["log","exp","inv","sqrt"]))

In [ ]:
for _ in range(6):
    qg_age.fit(df, threads=7)
    ql.update(qg_age.best())

In [ ]:
qg_age[0].sympify(3, symbolic_lr=False)

In [ ]:
print(qg_age[0].sympify(2))

In [ ]:
for _ in range(10000):
    qg_age[0].fit(df)

In [ ]:
qg_sc[0].plot_roc_curve(df)

# Prepare dataset for Cox regression

Having determined that $1/E$ and $1/C$ are closer associated with risk of death than $E$ and $C$ directly, we construct a data set with each of these values together with each patient's age.

In [ ]:
df_eng = pd.DataFrame({
    "TIME": df["TIME"],
    "Event": df["Event"],
    "exp_A": np.exp(0.056*df["Age"]),
    "inv_E": 100/df["EF"],
    "inv_C": 1/df["Creatinine"],
})

df_n_eng = pd.DataFrame({
    "TIME": df["TIME"],
    "Event": df["Event"],
    "A": df["Age"],
    "E": df["EF"],
    "C": df["Creatinine"],
})

# Cox regression

We then run a Cox regression on the resulting data set with only these covariates. Table \ref{tab:sr_cox} shows the coefficients (coef), the hazard ratios (HR) and the p-values for each of the risk factors. ($A$ is the age, $E$ is the ejection fraction, and $C$ is serum creatinine.)

In [ ]:
cph_eng = CoxPHFitter()
cph_eng.fit(df_eng, duration_col='TIME', event_col='Event')

In [ ]:
cph_eng.print_summary(columns=["coef","exp(coef)","exp(coef) lower 95%","exp(coef) upper 95%", "z", "p"], decimals=3)

In [ ]:
cph_eng.plot()

## Compare with Cox on unmodified features
The aim of this study is to investigate if functionally modified covariates (identified by symbolic regression) improves the predictive performance of Cox regression. We therefore fitted a Cox regression model on the same three covariates used above, but in unmodified form. 

In [ ]:
cph_n_eng = CoxPHFitter()
cph_n_eng.fit(df_n_eng, duration_col='TIME', event_col='Event')

cph_n_eng.print_summary(columns=["coef","exp(coef)","exp(coef) lower 95%","exp(coef) upper 95%", "z", "p"], decimals=3)

In [ ]:
cph_n_eng.print_summary(columns=["coef","exp(coef)","exp(coef) lower 95%","exp(coef) upper 95%", "z", "p"], decimals=3, style="latex")

In [ ]:
cph_n_eng.plot()

## ROC / AUC

Test the discrimination ability of the two models with a ROC curve and the Area Under the Curve (AUC).

This AUC value can be interpreted as the models' ability to correctly predict death within 285 days.

In [ ]:
feyn.plots.plot_roc_curve(df_eng["Event"], cph_eng.predict_cumulative_hazard(df_eng, times=[285]).T, label="Transformed covariates")
feyn.plots.plot_roc_curve(df_n_eng["Event"], cph_n_eng.predict_cumulative_hazard(df_n_eng, times=[285]).T, label="Untransformed covariates")

# Compare with 

Not part of the paper but for the curious we also compare with teh same feature set as Zahid et al.

In [ ]:
df_z_trans = pd.DataFrame({
    "TIME": df["TIME"],
    "Event": df["Event"],
    "exp_A": np.exp(0.056*df["Age"]),
    "inv_E": 100/df["EF"],
    "inv_C": 1/df["Creatinine"],
    "Sodium":  df["Sodium"],
    "Anaemia":  df["Anaemia"],
    "BP": df["BP"]
})

df_z_untrans = pd.DataFrame({
    "TIME": df["TIME"],
    "Event": df["Event"],
    "A": df["Age"],
    "E": df["EF"],
    "C": df["Creatinine"],
    "Sodium":  df["Sodium"],
    "Anaemia":  df["Anaemia"],
    "BP": df["BP"]
})

In [ ]:
cph_z_trans = CoxPHFitter()
cph_z_trans.fit(df_z_trans, duration_col='TIME', event_col='Event')
cph_z_trans.print_summary(columns=["coef","exp(coef)","exp(coef) lower 95%","exp(coef) upper 95%", "z", "p"], decimals=3)

In [ ]:
cph_z_untrans = CoxPHFitter()
cph_z_untrans.fit(df_z_untrans, duration_col='TIME', event_col='Event')
cph_z_untrans.print_summary(columns=["coef","exp(coef)","coef lower 95%","coef upper 95%", "z", "p"], decimals=3)

In [ ]:
feyn.plots.plot_roc_curve(df_z_trans["Event"], cph_z_trans.predict_cumulative_hazard(df_z_trans, times=[285]).T, label="Zahid features - transformed covariates")
feyn.plots.plot_roc_curve(df_z_untrans["Event"], cph_z_untrans.predict_cumulative_hazard(df_z_untrans, times=[285]).T, label="Zahid features - untransformed covariates")